# Setup Env

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
con_dev = ['MLCim', 'MLFaren', 'MLHabibi']

In [3]:
base_dir = '/content/drive/MyDrive/'+con_dev[2]+'/ChaMi-ML/Train'
%cd '$base_dir'

/content/drive/MyDrive/MLHabibi/ChaMi-ML/Train


In [ ]:
# import the required libraries
import data, model, datetime, time, pathlib, os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.metrics import Recall, Precision

# Get Data

In [ ]:
# set execute permission to .sh file
!chmod u+r+x ./GetUnzipImg.sh

# get data train and test
train_data, test_data = data.train_test_split()

# Get Model

In [ ]:
# instantiate embedding and its summary
embedding = model.create_embedding()
embedding.summary()

In [ ]:
# instantiate siamese model and its summary
my_model = model.create_siamese_model()
my_model.summary()

In [ ]:
# Plot embedding
plt.figure(figsize=(25,10))
plot_model(embedding, to_file='embedding.png', 
           show_shapes=True, rankdir='LR', 
           show_layer_names=False,
           dpi=1000)
img = plt.imread('embedding.png')
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Plot siamese model
plt.figure(figsize=(10,8))
plot_model(model, to_file='model.png',
           show_shapes=True, rankdir='LR',
           show_layer_names=False,
           dpi=100)
img = plt.imread('model.png')
plt.imshow(img)
plt.axis('off')
plt.show()

# Train Model

## Set optimizer, loss, and logs

In [ ]:
# set optimizer and loss function
optimizer = tf.keras.optimizers.Adam(1e-4)
binary_loss = tf.losses.BinaryCrossentropy()

# Set metrics
train_loss_metric = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
val_loss_metric = tf.keras.metrics.Mean('val_loss', dtype=tf.float32)
train_acc_metric = tf.keras.metrics.BinaryAccuracy('train_accuracy')
val_acc_metric = tf.keras.metrics.BinaryAccuracy('val_accuracy')

In [ ]:
# Set up summary writers to write the summaries to disk in a different logs directory
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
# Training checkpoints
if os.path.isdir('training_checkpoints'):
    !rm -r training_checkpoints
!mkdir training_checkpoints
checkpoint_directory = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_directory, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=my_model)

## Training loop, train and test step with tf function decorator

In [ ]:
# Train step function with tf function to ran TensorFlow eagerly
@tf.function
def train_step(batch):
  with tf.GradientTape() as tape:
    # Anchor and pos/neg images
    X = batch[:-1]
    # Label
    y = batch[-1]

    # Forward prop
    ypred = my_model(X, training=True)
    # Calculate the loss
    loss = binary_loss(y, ypred)
  
  # Calculate the gradients
  grad = tape.gradient(loss, my_model.trainable_variables)
  # Calculate updated weights and apply to model
  optimizer.apply_gradients(zip(grad, my_model.trainable_variables))
  train_loss_metric(loss)
  train_acc_metric.update_state(y, ypred)

  return loss

In [ ]:
# do the same w the evaluation step
@tf.function
def test_step(batch):
  # Anchor and pos/neg images
  X = batch[:-1]
  # Label
  y = batch[-1]
  
  # Make prediction
  ypred = my_model(X, training=False)
  loss = binary_loss(y, ypred)
  val_loss_metric(loss)
  val_acc_metric.update_state(y, ypred)

In [ ]:
# Train loop function
def train_model(train_data, val_data, epochs):
  # Loop epoch
  for epoch in range(1, epochs+1):
    print(f'\nEpoch {epoch}/{epochs}')

    start_time = time.time()

    # Loop batch
    for step, train_batch in enumerate(train_data):
      # Run train step
      loss_train = train_step(train_batch)
      # Log every 7 batches.
      if step % 7 == 0:
        print(
            "Training loss (for one batch) at step %d: %.4f"
            % (step+7, float(loss_train))
        )
        print("Seen so far: %d samples" % ((step+7) * 16))
    
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    with train_summary_writer.as_default():
      tf.summary.scalar('loss', train_loss_metric.result(), step=epoch)
      tf.summary.scalar('accuracy', train_acc, step=epoch)

    # Run a validation loop at the end of each epoch.
    for val_batch in val_data:
      test_step(val_batch)
    
    val_acc = val_acc_metric.result()

    with test_summary_writer.as_default():
      tf.summary.scalar('loss', val_loss_metric.result(), step=epoch)
      tf.summary.scalar('accuracy', val_acc, step=epoch)

    print("Validation acc: %.4f" % (float(val_acc),))

    print("Time taken: %.2fs" % (time.time() - start_time))
    
    # Reset metrics every epoch
    train_loss_metric.reset_states()
    train_acc_metric.reset_states()
    val_loss_metric.reset_states()
    val_acc_metric.reset_states()

    # Checkpoints
    if epoch % 5 == 0:
      checkpoint.save(file_prefix=checkpoint_prefix)

## Train the model

In [ ]:
# Config training model
EPOCHS = 15

# Train model
train_model(train_data, test_data, EPOCHS) 

# Evaluation

## Evaluation Loss

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/gradient_tape/

## Evaluation Metrics

In [ ]:
# get a batch of test data
test_input, test_validation, y_test = test_img.as_numpy_iterator().next()

In [ ]:
# predict
y_pred = my_model.predict([test_input, test_validation])



> Compare actual and pred img



In [ ]:
# processing the sigmoid result
[1 if pred > 0.5 else 0 for pred in y_pred]

In [ ]:
y_test

> Calculate metrics

In [ ]:
# instantiating recall and precision metrics
recall = Recall()
precision = Precision()

recall.update_state(y_test, y_pred)
precision.update_state(y_test, y_pred)

print('Recall      : '.format(recall.result().numpy()))
print('Precision   : '.format(precision.result().numpy()))

# Prepare for Deployment

## Tensorflow js

> save model

In [ ]:
tfjs_dir = '/content/drive/MyDrive/'+con_dev[0]+'/ChaMi-ML/Models/tfjs'
tf.keras.experimental.export_saved_model(my_model, tfjs_dir)

> convert model *(this can be done on the command line)*

In [ ]:
!tensorflowjs_converter \
    --input_format=keras_saved_model \
    '$tfjs_dir' \
    '$tfjs_dir'/mymodeltfjs

In [ ]:
# for .h5 model
my_model.save(tfjs_dir+'/my_model.h5')
!tensorflowjs_converter \
    --input_format=keras \
    '$tfjs_dir'/my_model.h5 \
    '$tfjs_dir'/modeltfjs-h5

In [ ]:
!saved_model_cli show --dir '$tfjs_dir' --tag_set serve --signature_default

## Tensorflow lite

In [ ]:
# generate save model
tflite_dir = '/content/drive/MyDrive/'+con_dev[2]+'/ChaMi-ML/Models/tflite'
tf.saved_model.save(my_model, tflite_dir)

> standard

In [ ]:
# convert model
converter = tf.lite.TFLiteConverter.from_saved_model(tflite_dir)
tflite_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path(tflite_dir+'/my_model.tflite')
tflite_model_file.write_bytes(tflite_model)

> quantize

In [ ]:
# with quantization
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path(tflite_dir+'/my_quantmodel.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

> select ops

In [ ]:
# optimize with tf select ops
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                        tf.lite.OpsSet.SELECT_TF_OPS]
tflite_ops_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path(tflite_dir+'/my_opsmodel.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

## Tensorflow serving

> using tensorflow model and docker

In [ ]:
tfserving_dir = '/content/drive/MyDrive/'+con_dev[2]+'/ChaMi-ML/Models/tfserving'
tf.saved_model.save(model, tfserving_dir)

> using tensorFlow Serving Distribution URI as a Package Source

In [ ]:
version = 1
export_path = os.path.join(tfserving_dir, str(version))
if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

my_model.save(export_path, save_format="tf")

print('\nexport_path = {}'.format(export_path))
!ls -l {export_path}

In [ ]:
# show saved model
!saved_model_cli show --dir {export_path} --all